In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
import gc
import joblib
import pandas as pd
import numpy as np
#import tensorflow as tf

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing

from keras import layers
from keras import optimizers
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras import utils

/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/share/plaidml/experimental.json
/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/local/share/plaidml/experimental.json
/usr/local/share/plaidml/experimental.json
/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/share/plaidml/config.json
/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/local/share/plaidml/config.json
/usr/local/share/plaidml/config.json


Using plaidml.keras.backend backend.


In [3]:
def create_model(data, catcols):    
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2), 50))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=y)
    return model

In [4]:
train = pd.read_csv("./data_cat_maxAUC_0.78/train.csv")
test = pd.read_csv("./data_cat_maxAUC_0.78/test.csv")
sample = pd.read_csv("./data_cat_maxAUC_0.78/sample_submission.csv")

In [5]:
test["target"] = -1
data = pd.concat([train, test]).reset_index(drop=True)

features = [x for x in train.columns if x not in ["id", "target"]]

for feat in features:
    lbl_enc = preprocessing.LabelEncoder()
    data[feat] = lbl_enc.fit_transform(data[feat].fillna("-1").astype(str).values)

In [6]:
data

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,1,1,1,1,1,3,5,4,6,...,28,3,1,4,3,21,57,6,6,0
1,1,2,2,1,1,2,3,4,1,0,...,2113,3,3,6,5,24,151,7,10,0
2,2,1,2,1,1,1,3,0,4,1,...,0,3,0,3,14,16,106,5,12,0
3,3,0,1,1,1,1,3,1,4,4,...,2168,1,5,5,1,3,0,3,6,0
4,4,1,0,1,2,1,3,6,4,3,...,1748,3,3,2,8,3,51,5,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,1,1,2,2,1,3,5,1,3,...,616,2,2,6,14,18,104,2,4,-1
999996,999996,1,1,1,1,1,3,2,3,6,...,1987,1,1,0,14,24,80,1,8,-1
999997,999997,1,1,2,2,1,3,6,4,0,...,632,1,3,1,13,16,74,1,9,-1
999998,999998,1,1,1,1,2,3,1,3,3,...,1988,3,5,5,8,12,12,2,3,-1


In [7]:
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)
test_data = [test.loc[:, features].values[:, k] for k in range(test.loc[:, features].values.shape[1])]

In [13]:
train.to_csv('data-CATII.csv.zip')

In [10]:
oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))

skf = StratifiedKFold(n_splits=50)
for train_index, test_index in skf.split(train, train.target.values):
    
    X_train, X_test = train.iloc[train_index, :], train.iloc[test_index, :]
    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train, y_test = X_train.target.values, X_test.target.values
    
    model = create_model(data, features)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    X_train = [X_train.loc[:, features].values[:, k] for k in range(X_train.loc[:, features].values.shape[1])]
    X_test = [X_test.loc[:, features].values[:, k] for k in range(X_test.loc[:, features].values.shape[1])]
    
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=5,
                                 verbose=1, mode='max', baseline=None, restore_best_weights=True)

    rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
                                      patience=3, min_lr=1e-6, mode='max', verbose=1)
    
    model.fit(X_train,
              utils.to_categorical(y_train),
              validation_data=(X_test, utils.to_categorical(y_test)),
              verbose=1,
              batch_size=1024,
              callbacks=[es, rlr],
              epochs=100
             )
    
    valid_fold_preds = model.predict(X_test)[:, 1]
    test_fold_preds = model.predict(test_data)[:, 1]
    oof_preds[test_index] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    
    print(metrics.roc_auc_score(y_test, valid_fold_preds))
    K.clear_session()

Train on 588000 samples, validate on 12000 samples
Epoch 1/100
323584/588000 [===============>..............] - ETA: 45s - loss: 0.5091

KeyboardInterrupt: 